In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import sys
sys.path.extend(['..'])
from utils import Config
from data import LSTMCSVDataset
import copy 
from tqdm import tqdm
import os
import json
from datetime import datetime

In [2]:
class LSTMRegressor(nn.Module):
    def __init__(self, input_size, output_size):
        super(LSTMRegressor,self).__init__()
        self.input_size = input_size
        self.output_size = output_size

        self.lstm_1 = nn.LSTM(self.input_size, 100, batch_first=True)
        self.fc1 = nn.Linear(1000, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, output_size)
        self.h1 = self.init_hidden(batch_size=Config['batch_size'],device='cuda:0')

    def init_hidden(self, batch_size, device='cpu'):
        return (torch.zeros(1, batch_size , 100).to(device), torch.zeros(1, batch_size , 100).to(device))
    
    def forward(self, x, device='cpu'):
        out, self.h1 = self.lstm_1(x, self.h1)
#         print(out.shape)
        out = out.reshape(out.shape[0],-1)
        out = nn.Tanh()(self.bn2(self.fc1(out)))
        out = nn.Tanh()(self.bn3(self.fc2(out)))
        out = nn.Tanh()(self.bn4(self.fc3(out)))
        out = nn.Tanh()(self.fc4(out))
#         print(out.shape)
        return out

In [3]:
def train(dataloader, dataset_sizes, model, criterion, optimizer, device, num_epochs=Config['num_epochs'], batch_size=Config['batch_size']):
    print(Config)

    start_time = datetime.now()
    print("Start Time =", start_time)

    model.to(device)

    best_loss = 100.0
    best_wts = copy.deepcopy(model.state_dict())
    dummy_input = None

    if Config['tensorboard_log']:
        writer = SummaryWriter(Config['model_path'])
    
    for epoch in range(num_epochs):
        for phase in ['train', 'valid']:
            if phase=='train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            print(dataloader[phase])
            for inputs, labels in tqdm(dataloader[phase]):
                # if inputs.shape[0]!=Config['batch_size']:
                    # continue
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase=='train'):
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    model.h1= model.init_hidden(batch_size=inputs.shape[0], device='cuda:0')

                    if dummy_input is None:
                        dummy_input = inputs

                    outputs = model(inputs)
#                     print(outputs.shape, labels[:,-1,:].shape)
                    loss = criterion(outputs, labels[:,-1,:])
                    if Config['use_cuda']:
                        l2_regularization = torch.tensor(0.).cuda()
                    else:
                        l2_regularization = torch.tensor(0.)
                    
                    for param in model.parameters():
                        l2_regularization += torch.norm(param, 2)**2

                    loss += 1e-5 * l2_regularization
                    if phase =='train':
                        loss.backward()
                        optimizer.step()
                        model.h1[0].detach_()
                        model.h1[1].detach_()
                    running_loss += loss.item()*inputs.size(0)
            epoch_loss = running_loss/dataset_sizes[phase]
            print(f'Epoch {epoch} Loss: {epoch_loss:.4f}')
            if epoch_loss < best_loss and phase=='valid':
                best_wts = copy.deepcopy(model.state_dict())
                print(outputs[-1,:], labels[-1,-1,:])
            writer.add_scalar(phase+'_loss', epoch_loss, global_step=epoch)
        if (epoch+1)%5==0:
            model.h1 = model.init_hidden(batch_size=1, device='cuda:0')
            torch.onnx.export(model,
                      dummy_input[0].unsqueeze(0),
                      os.path.join(
                            Config['model_path'],
                            f'checkpoints/model_{epoch}.onnx'
                        ),
                      )
            torch.save(
                model.state_dict(),
                os.path.join(
                    Config['model_path'],
                    f'checkpoints/model_{epoch}.pth'
                )
            )
    print('Training ended')
    end_time = datetime.now()
    print("End Time =", end_time)
    print("Total Time =", end_time-start_time)
    torch.save(
        best_wts,
        os.path.join(
            Config['model_path'],
            'checkpoints/model_final.pth'
        )
    )
    model.load_state_dict(best_wts)
    torch.onnx.export(model,
                      dummy_input,
                      os.path.join(
                            Config['model_path'],
                            'checkpoints/model_final.onnx'
                      ),
                      export_params=True
                      )

In [28]:
Config['model_path'] = '../models/lstm_alldata_run1_continue2'

In [5]:
Config['dataset_path'] = os.path.join('..',Config['dataset_path'])

In [30]:

os.makedirs(Config['model_path'], exist_ok=True)
os.makedirs(os.path.join(Config['model_path'],'logs'),exist_ok=True)
os.makedirs(os.path.join(Config['model_path'],'checkpoints'),exist_ok=True)

dataset = LSTMCSVDataset(root_path=Config['dataset_path'])
# model = LSTMRegressor(input_size=19, output_size=7)

device = torch.device('cuda:0' if torch.cuda.is_available() and Config['use_cuda'] else 'cpu')

optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-6) #rmsprop, adam

criterion = torch.nn.MSELoss(reduction='sum')

ratio = [int(len(dataset)*0.9), len(dataset)-int(len(dataset)*0.9)]
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, ratio)

train_loader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size = Config['batch_size'],
                    shuffle=True,
                    num_workers = Config['num_workers']
                )

valid_loader = torch.utils.data.DataLoader(
                    valid_dataset,
                    batch_size = Config['batch_size'],
                    shuffle=False,
                    num_workers = Config['num_workers']
                )
dataloaders = {
    'train' : train_loader,
    'valid' : valid_loader
}

dataset_sizes = {
    'train' : len(train_dataset),
    'valid' : len(valid_dataset)
}

train(
    dataloaders, 
    dataset_sizes,
    model,
    criterion,
    optimizer,
    device
    )

  0%|                                                                                                                                      | 0/1390 [00:00<?, ?it/s]

{'dataset_path': '..\\Data_Exp/Exp0', 'model_path': '../models/lstm_alldata_run1_continue2', 'onnx_model_path': 'exp0_lstm_model_final.onnx', 'tensorboard_log': True, 'use_cuda': True, 'lr': 0.001, 'num_epochs': 1000, 'batch_size': 256, 'num_workers': 2, 'data_type': 'relative'}
Start Time = 2021-04-21 07:47:34.021422


  0%|                                                                                                                                       | 0/155 [00:00<?, ?it/s]

Epoch 0 Loss: 7.1618


  0%|                                                                                                                                      | 0/1390 [00:00<?, ?it/s]

...

Epoch 782 Loss: 3.8760


  0%|                                                                                                                                      | 0/1390 [00:00<?, ?it/s]

Epoch 782 Loss: 4.7947
tensor([ 0.0281,  0.6648,  0.3542, -1.0000, -0.5660, -0.0687,  0.9007],
       device='cuda:0') tensor([ 0.0411,  0.6675,  0.3487, -0.9979, -0.5593, -0.0647,  0.8906],
       device='cuda:0')


 27%|█████████████████████████████████                                                                                           | 370/1390 [00:11<00:30, 33.32it/s]


KeyboardInterrupt: 

In [7]:
class LSTMCSVDataset(torch.utils.data.Dataset):
    def __init__(self, root_path, output_type=Config['data_type'], look_back=10, step_value = 1):
        files = os.listdir(root_path)
        files = [f for f in files if f.endswith('.csv')]
        all_data=[]
       
        for file in files:
            csv_data = pd.read_csv(os.path.join(root_path,file)).iloc[:,1:41]
            
            csv_data['relativeHandRPosx'] = csv_data.headPosx-csv_data.handRPosx
            csv_data['relativeHandRPosy'] = csv_data['headPosy']-csv_data['handRPosy']
            csv_data['relativeHandRPosz'] = csv_data['headPosz']-csv_data['handRPosz']
            csv_data['relativeHandLPosx'] = csv_data['headPosx']-csv_data['handLPosx']
            csv_data['relativeHandLPosy'] = csv_data['headPosy']-csv_data['handLPosy']
            csv_data['relativeHandLPosz'] = csv_data['headPosz']-csv_data['handLPosz']
            csv_data['relativeTracker1Posx'] = csv_data['headPosx']-csv_data['tracker1Posx']
            csv_data['relativeTracker1Posy'] = csv_data['headPosy']-csv_data['tracker1Posy']
            csv_data['relativeTracker1Posz'] = csv_data['headPosz']-csv_data['tracker1Posz']
            
            data = np.array([csv_data])
            all_data.append(data)
        self.data = np.concatenate(all_data, axis=1).squeeze(0)
        self.scaler = MinMaxScaler(feature_range=(-1,1))
        self.scaler.fit(self.data)
        self.scaled_data = self.scaler.transform(self.data)
        self.lstm_data = []
        for i in range(len(self.scaled_data)-look_back-1):
            a = self.scaled_data[i:(i+look_back):step_value]
            self.lstm_data.append(a)
        self.lstm_data = np.array(self.lstm_data)
        self.output_type = output_type
        self.look_back = look_back
        self.step_value = step_value
    def __len__(self):
        return self.lstm_data.shape[0]
    
    def __getitem__(self, idx):
        row = self.lstm_data[idx]
        headPosx = row[:, 0]
        headPosy = row[:,1]
        headPosz = row[:,2]
        headRotx = row[:,3]
        headRoty = row[:,4]
        headRotz = row[:,5]
        headRotQx = row[:,6]
        headRotQy = row[:,7]
        headRotQz = row[:,8]
        headRotQw = row[:,9]

        handRPosx = row[:,10]
        handRPosy = row[:,11]
        handRPosz = row[:,12]
        handRRotx = row[:,13]
        handRRoty = row[:,14]
        handRRotz = row[:,15]
        handRRotQx = row[:,16]
        handRRotQy = row[:,17]
        handRRotQz = row[:,18]
        handRRotQw = row[:,19]

        handLPosx = row[:,20]
        handLPosy = row[:,21]
        handLPosz = row[:,22]
        handLRotx = row[:,23]
        handLRoty = row[:,24]
        handLRotz = row[:,25]
        handLRotQx = row[:,26]
        handLRotQy = row[:,27]
        handLRotQz = row[:,28]
        handLRotQw = row[:,29]

        tracker1Posx = row[:,30]
        tracker1Posy = row[:,31]
        tracker1Posz = row[:,32]
        tracker1Rotx = row[:,33]
        tracker1Roty = row[:,34]
        tracker1Rotz = row[:,35]
        tracker1RotQx = row[:,36]
        tracker1RotQy = row[:,37]
        tracker1RotQz = row[:,38]
        tracker1RotQw = row[:,39]

        relativeHandRPosx = row[:,40]
        relativeHandRPosy = row[:,41]
        relativeHandRPosz = row[:,42]
        relativeHandLPosx = row[:,43]
        relativeHandLPosy = row[:,44]
        relativeHandLPosz = row[:,45]
        relativeTracker1Posx = row[:,46]
        relativeTracker1Posy = row[:,47]
        relativeTracker1Posz = row[:,48]

        if self.output_type=='euler':
            return (
                torch.FloatTensor(np.stack([headPosx, headPosy, headPosz, headRotx, headRoty, headRotz,
                    handRPosx, handRPosy, handRPosz, handRRotx, handRRoty, handRRotz,
                    handLPosx, handLPosy, handLPosz, handLRotx, handLRoty, handLRotz,
                    ],axis=-1)),
                torch.FloatTensor(np.stack([tracker1Posx, tracker1Posy, tracker1Posz, tracker1Rotx, tracker1Roty, 
                    tracker1Rotz],axis=-1))
            )
        elif self.output_type=='quaternion':
            return (
                torch.FloatTensor(np.stack([headPosx, headPosy, headPosz, headRotQx, headRotQy, headRotQz, headRotQw,
                    handRPosx, handRPosy, handRPosz, handRRotQx, handRRotQy, handRRotQz, handRRotQw,
                    handLPosx, handLPosy, handLPosz, handLRotQx, handLRotQy, handLRotQz, handLRotQw,
                    ],axis=-1)),
                torch.FloatTensor(np.stack([tracker1Posx, tracker1Posy, tracker1Posz, tracker1RotQx, tracker1RotQy, 
                    tracker1RotQz, tracker1RotQw],axis=-1))
            )
        elif self.output_type=='both':
            return (
                torch.FloatTensor(np.stack([headPosx, headPosy, headPosz, headRotx, headRoty, headRotz,headRotQx, headRotQy, headRotQz, headRotQw,
                    handRPosx, handRPosy, handRPosz, handRRotx, handRRoty, handRRotz, handRRotQx, handRRotQy, handRRotQz, handRRotQw,
                    handLPosx, handLPosy, handLPosz, handLRotx, handLRoty, handLRotz, handLRotQx, handLRotQy, handLRotQz, handLRotQw
                    ],axis=-1)),
                torch.FloatTensor(np.stack([tracker1Posx, tracker1Posy, tracker1Posz, tracker1Rotx, tracker1Roty, 
                    tracker1Rotz, tracker1RotQx, tracker1RotQy, tracker1RotQz, tracker1RotQw],axis=-1))
            )
        elif self.output_type=='relative' or self.output_type=='relative_svm' or self.output_type=='hacklstm':
            return (
                torch.FloatTensor(np.stack([headPosy, headRotQx, headRotQy, headRotQz, headRotQw,
                    relativeHandRPosx, relativeHandRPosy, relativeHandRPosz, handRRotQx, handRRotQy, handRRotQz, handRRotQw,
                    relativeHandLPosx, relativeHandLPosy, relativeHandLPosz, handLRotQx, handLRotQy, handLRotQz, handLRotQw
                    ],axis=-1)),
                torch.FloatTensor(np.stack([relativeTracker1Posx, relativeTracker1Posy, relativeTracker1Posz,
                    tracker1RotQx, tracker1RotQy, tracker1RotQz, tracker1RotQw],axis=-1))
            )
import pandas as pd
import numpy as np
def compute_accuracy(model, csv_file, scaler_file, out_file='out.csv', device='cuda:0'):
    scaler = json.load(open(scaler_file,'r'))
    writer=open(out_file,'w')
    csv_data = pd.read_csv(csv_file).iloc[:,1:41]
            
    csv_data['relativeHandRPosx'] = csv_data.headPosx-csv_data.handRPosx
    csv_data['relativeHandRPosy'] = csv_data['headPosy']-csv_data['handRPosy']
    csv_data['relativeHandRPosz'] = csv_data['headPosz']-csv_data['handRPosz']
    csv_data['relativeHandLPosx'] = csv_data['headPosx']-csv_data['handLPosx']
    csv_data['relativeHandLPosy'] = csv_data['headPosy']-csv_data['handLPosy']
    csv_data['relativeHandLPosz'] = csv_data['headPosz']-csv_data['handLPosz']
    csv_data['relativeTracker1Posx'] = csv_data['headPosx']-csv_data['tracker1Posx']
    csv_data['relativeTracker1Posy'] = csv_data['headPosy']-csv_data['tracker1Posy']
    csv_data['relativeTracker1Posz'] = csv_data['headPosz']-csv_data['tracker1Posz']
    keys = list(csv_data.keys())
#     print(keys)
    data = np.array([csv_data]).squeeze(0)
    scalerDict = dict()
    for param in scaler['scalers']:
        scalerDict.update({param['type']: [param['min'], param['scale']]})
    model.eval()
    model.to(device)
    input_keys = ['headPosy', 'headRotQx', 'headRotQy', 'headRotQz', 'headRotQw',
                    'relativeHandRPosx', 'relativeHandRPosy', 'relativeHandRPosz', 'handRRotQx', 'handRRotQy', 'handRRotQz', 'handRRotQw',
                    'relativeHandLPosx', 'relativeHandLPosy', 'relativeHandLPosz', 'handLRotQx', 'handLRotQy', 'handLRotQz', 'handLRotQw'
                    ]
    output_keys = ['relativeTracker1Posx', 'relativeTracker1Posy', 'relativeTracker1Posz',
                    'tracker1RotQx', 'tracker1RotQy', 'tracker1RotQz', 'tracker1RotQw']
    writer.write(','.join(input_keys)+','+','.join(output_keys)+','+','.join(['pred'+k for k in output_keys])+'\n')
#     print(','.join(input_keys)+','+','.join(output_keys)+','+','.join(['pred'+k for k in output_keys])+'\n')
    stats = {}
    denorm_stats = {}
    for key in output_keys:
        stats[key]={
            'running_mse':0,
            'running_r2':0,
            'min_mse':1000,
            'max_mse':-1000,
            'min_r2':1000,
            'max_r2':-1000
        }
        denorm_stats[key]={
            'running_mse':0,
            'running_r2':0,
            'min_mse':1000,
            'max_mse':-1000,
            'min_r2':1000,
            'max_r2':-1000
        }
    for i in range(data.shape[0]):
#         print(data[i].shape)
        if i<10:
            a = [data[i]]*10
        else:
            a = data[i-10:i:1]
#         print(a)
        a = np.array(a)
#         print(a.shape)
        headPosy = a[:,keys.index('headPosy')]
        headRotQx = a[:,keys.index('headRotQx')]
        headRotQy = a[:,keys.index('headRotQy')]
        headRotQz = a[:,keys.index('headRotQz')]
        headRotQw = a[:,keys.index('headRotQw')]
        relativeHandRPosx = a[:,keys.index('relativeHandRPosx')]
        relativeHandRPosy = a[:,keys.index('relativeHandRPosy')]
        relativeHandRPosz = a[:,keys.index('relativeHandRPosz')]
        handRRotQx = a[:,keys.index('handRRotQx')]
        handRRotQy = a[:,keys.index('handRRotQy')]
        handRRotQz = a[:,keys.index('handRRotQz')]
        handRRotQw = a[:,keys.index('handRRotQw')]
        relativeHandLPosx = a[:,keys.index('relativeHandLPosx')]
        relativeHandLPosy = a[:,keys.index('relativeHandLPosy')]
        relativeHandLPosz = a[:,keys.index('relativeHandLPosz')]
        handLRotQx = a[:,keys.index('handLRotQx')]
        handLRotQy = a[:,keys.index('handLRotQy')]
        handLRotQz = a[:,keys.index('handLRotQz')]
        handLRotQw = a[:,keys.index('handLRotQw')]
        
        input_tensor = np.vstack([headPosy, headRotQx, headRotQy, headRotQz, headRotQw, 
                                 relativeHandRPosx, relativeHandRPosy, relativeHandRPosz,
                                 handRRotQx, handRRotQy, handRRotQz, handRRotQw,
                                 relativeHandLPosx, relativeHandLPosy, relativeHandLPosz,
                                 handLRotQx, handLRotQy, handLRotQz, handLRotQw]).T
        
        input_normalized_tensor = input_tensor.copy()
        
#         print(input_normalized_tensor.shape)
        for idx, key in enumerate(input_keys):
            input_normalized_tensor[:,idx] = input_normalized_tensor[:,idx] * scalerDict[key][1] + scalerDict[key][0]
        #a[key] = a[key] * scalerDict[key][1] + scalerDict[key][0]
        
#         print(input_normalized_tensor.T)
        relativeTracker1Posx = a[:,keys.index('relativeTracker1Posx')]
        relativeTracker1Posy = a[:,keys.index('relativeTracker1Posy')]
        relativeTracker1Posz = a[:,keys.index('relativeTracker1Posz')]
        tracker1RotQx = a[:,keys.index('tracker1RotQx')]
        tracker1RotQy = a[:,keys.index('tracker1RotQy')]
        tracker1RotQz = a[:,keys.index('tracker1RotQz')]
        tracker1RotQw = a[:,keys.index('tracker1RotQw')]
        
        output_tensor= np.vstack([relativeTracker1Posx, relativeTracker1Posy, relativeTracker1Posz,
                    tracker1RotQx, tracker1RotQy, tracker1RotQz, tracker1RotQw]).T
        output_tensor = np.array([output_tensor[-1]])
        output_normalized_tensor = output_tensor.copy()
        
        
#         print(scalerDict)
        for idx, key in enumerate(output_keys):
            output_normalized_tensor[:,idx] = output_normalized_tensor[:, idx] * scalerDict[key][1] + scalerDict[key][0]
        with torch.set_grad_enabled(False):
            model.h1 = model.init_hidden(1,device)
            predicted_normalized_tensor = model(torch.FloatTensor(input_normalized_tensor).unsqueeze(0).to(device)).detach().cpu().numpy()
            predicted_tensor = predicted_normalized_tensor.copy()
            for idx, key in enumerate(output_keys):
                predicted_tensor[:,idx] = (predicted_tensor[:,idx] - scalerDict[key][0]) / scalerDict[key][1]
#             print(predicted_tensor)
#             print(output_tensor)
#             print(predicted_normalized_tensor)
#             print(output_normalized_tensor)
#             print(scaler)
            writer.write(','.join([str(val) for val in input_tensor[0]])+','+','.join([str(val) for val in output_tensor[0]])+','+','.join([str(val) for val in predicted_tensor[0]])+'\n')
#             print(','.join([str(val) for val in input_tensor[0]])+','+','.join([str(val) for val in predicted_tensor[0]]))
            loss = torch.nn.MSELoss(reduction='none')(torch.FloatTensor(predicted_normalized_tensor), torch.FloatTensor(output_normalized_tensor)).cpu().numpy()
            denorm_loss = torch.nn.MSELoss(reduction='none')(torch.FloatTensor(predicted_tensor), torch.FloatTensor(output_tensor)).cpu().numpy()
#             print(loss)
            for idx, key in enumerate(output_keys):
#                 score = r2_score(predicted_normalized_tensor[:,idx], output_normalized_tensor[:,idx])
                stats[key]['running_mse']+=loss[0,idx]
#                 stats[key]['running_r2']+=score
                if loss[0,idx]<stats[key]['min_mse']:
                    stats[key]['min_mse']=loss[0,idx]
                if loss[0,idx]>stats[key]['max_mse']:
                    stats[key]['max_mse']=loss[0,idx]
#                 if score<stats[key]['min_r2']:
#                     stats[key]['min_r2']=score
#                 if score>stats[key]['max_r2']:
#                     stats[key]['max_r2']=score
                    
#                 score = r2_score(predicted_tensor[:,idx], output_tensor[:,idx])
                denorm_stats[key]['running_mse']+=denorm_loss[0,idx]
#                 denorm_stats[key]['running_r2']+=score
                if denorm_loss[0,idx]<denorm_stats[key]['min_mse']:
                    denorm_stats[key]['min_mse']=denorm_loss[0,idx]
                if denorm_loss[0,idx]>denorm_stats[key]['max_mse']:
                    denorm_stats[key]['max_mse']=denorm_loss[0,idx]
#                 if score<denorm_stats[key]['min_r2']:
#                     denorm_stats[key]['min_r2']=score
#                 if score>stats[key]['max_r2']:
#                     denorm_stats[key]['max_r2']=score
                
#         break
    for idx, key in enumerate(output_keys):
        stats[key]['running_mse']/=data.shape[0]
        stats[key]['running_r2']/=data.shape[0]
        denorm_stats[key]['running_mse']/=data.shape[0]
        denorm_stats[key]['running_r2']/=data.shape[0]
    writer.close()
    print(stats)
    print()
    print(denorm_stats)

In [196]:
compute_accuracy(model, r'C:\Users\CSCI-538-HP-Z240-lll\Desktop\UnityProjects\XR-Interaction-Toolkit-Examples\ML\Data_Exp\Exp0\BeatSaber_1_processed.csv',r'C:\Users\CSCI-538-HP-Z240-lll\Desktop\UnityProjects\XR-Interaction-Toolkit-Examples\ML\models\lstm_alldata_run7\scaler.json')

{'relativeTracker1Posx': {'running_mse': 0.0005360931217298857, 'running_r2': 0.0, 'min_mse': 2.7200464e-13, 'max_mse': 0.022085233, 'min_r2': 1000, 'max_r2': -1000}, 'relativeTracker1Posy': {'running_mse': 0.0001944142525954155, 'running_r2': 0.0, 'min_mse': 1.4210855e-14, 'max_mse': 0.027564276, 'min_r2': 1000, 'max_r2': -1000}, 'relativeTracker1Posz': {'running_mse': 0.0001927561358632688, 'running_r2': 0.0, 'min_mse': 2.220446e-14, 'max_mse': 0.033536203, 'min_r2': 1000, 'max_r2': -1000}, 'tracker1RotQx': {'running_mse': 4.108150435839741e-06, 'running_r2': 0.0, 'min_mse': 2.2737368e-13, 'max_mse': 0.0050250217, 'min_r2': 1000, 'max_r2': -1000}, 'tracker1RotQy': {'running_mse': 0.0006010004392077814, 'running_r2': 0.0, 'min_mse': 8.881784e-16, 'max_mse': 0.030070955, 'min_r2': 1000, 'max_r2': -1000}, 'tracker1RotQz': {'running_mse': 0.00015760533449687855, 'running_r2': 0.0, 'min_mse': 7.214229e-13, 'max_mse': 0.015139171, 'min_r2': 1000, 'max_r2': -1000}, 'tracker1RotQw': {'runnin

In [162]:
(-0.999 - -0.9979719963357392) / 0.005550011628897741

-0.18522549734999008

In [149]:
torch.nn.MSELoss(reduction='none')(torch.FloatTensor([[-0.05364176,0.7452757,0.39267138,-1.,-0.05440243,-0.10438076, -0.9996881 ]]),torch.FloatTensor([[-0.07013874,0.75882053,0.39598479,-0.99802646,-0.03897533,-0.10258538,-0.99907593]]))

tensor([[2.7215e-04, 1.8346e-04, 1.0979e-05, 3.8950e-06, 2.3800e-04, 3.2234e-06,
         3.7472e-07]])

In [9]:
import sklearn
from sklearn.metrics import r2_score

r2_score([ 8.2639e-02,  7.1756e-01,  4.0250e-01,  -9.5940e-01, 3.6648e-02,
        -9.4972e-01,  4.9748e-04],[ 0.0707,  0.7066,  0.4055,  0.9897, -0.0197, -0.9630,  0.0443])

-0.5570027868390335

In [22]:
import sklearn
from sklearn.metrics import r2_score

def calc_accuracy(model, dataloader, device='cuda:0'):
    running_acc = 0.0
    running_total = 0
    model.eval()
    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        model.h1= model.init_hidden(batch_size=inputs.shape[0], device='cuda:0')
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
#         print(outputs.shape, labels.shape)
        acc = r2_score(outputs.detach().cpu().numpy(), labels[:,-1,:].detach().cpu().numpy())
        running_acc+=acc
        running_total+=1
        print(outputs, labels[:,-1,:])
    print(running_acc/running_total)
    print(running_acc)

In [23]:
calc_accuracy(model, valid_loader)

  6%|███████▎                                                                                                                       | 9/155 [00:05<01:06,  2.18it/s]

tensor([[-0.0510,  0.7028,  0.3873,  ...,  0.3536, -0.0613, -0.9947],
        [-0.0604,  0.6524,  0.4150,  ..., -0.1771, -0.0849,  0.9970],
        [-0.0401,  0.7170,  0.3866,  ...,  0.1277, -0.0590,  0.9990],
        ...,
        [ 0.0524,  0.6725,  0.3320,  ...,  0.1591, -0.0875,  0.9459],
        [-0.1213,  0.5566,  0.4282,  ..., -0.0977, -0.0532,  0.9999],
        [-0.0368,  0.6340,  0.3970,  ...,  0.1618, -0.0531,  0.9916]],
       device='cuda:0') tensor([[-0.0561,  0.6794,  0.3861,  ...,  0.3277, -0.0539, -0.9276],
        [-0.0326,  0.6175,  0.3984,  ..., -0.1803, -0.0795,  0.9947],
        [-0.0160,  0.7419,  0.3600,  ...,  0.1265, -0.0243,  0.9817],
        ...,
        [ 0.0514,  0.6694,  0.3247,  ...,  0.1700, -0.0945,  0.9739],
        [-0.1691,  0.5165,  0.4304,  ..., -0.0760, -0.0127,  0.9983],
        [-0.0401,  0.5907,  0.3989,  ...,  0.1895, -0.0331,  0.9674]],
       device='cuda:0')
tensor([[-0.0465,  0.6690,  0.4282,  ..., -0.0922, -0.0599,  0.9927],
        [-0.38

 16%|████████████████████▎                                                                                                         | 25/155 [00:05<00:15,  8.38it/s]

tensor([[-0.1305,  0.5348,  0.2042,  ...,  0.9392, -0.1036,  0.2541],
        [-0.0794,  0.7130,  0.3922,  ..., -0.0719, -0.0743,  0.7817],
        [-0.0855,  0.6816,  0.3978,  ..., -0.0790, -0.0658,  0.9998],
        ...,
        [-0.2146,  0.6827,  0.4098,  ..., -0.0494, -0.0298,  0.9996],
        [-0.1003,  0.6841,  0.3973,  ..., -0.0325,  0.6840, -0.0618],
        [ 0.0017,  0.7365,  0.3930,  ..., -0.0985,  0.9726, -0.1391]],
       device='cuda:0') tensor([[-0.1508,  0.5229,  0.2118,  ...,  0.9463, -0.0851,  0.3122],
        [-0.0580,  0.7038,  0.4150,  ..., -0.0625, -0.0421,  0.9996],
        [-0.1135,  0.6242,  0.4025,  ..., -0.0389, -0.0362,  0.9989],
        ...,
        [-0.2145,  0.6705,  0.4286,  ..., -0.0620, -0.0115,  0.9983],
        [-0.1099,  0.6960,  0.3912,  ..., -0.0454,  0.6658, -0.0808],
        [-0.0078,  0.7427,  0.3788,  ..., -0.1038,  0.9663, -0.1258]],
       device='cuda:0')
tensor([[-2.1776e-01,  5.9279e-01,  3.5417e-01,  ...,  9.2944e-04,
          7.8874e

 26%|████████████████████████████████▌                                                                                             | 40/155 [00:06<00:06, 17.49it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 153/155 [00:07<00:00, 73.12it/s]

tensor([[ 0.3176,  0.1746,  0.4067,  ..., -0.1929, -0.1303, -0.9972],
        [-0.0207,  0.7581,  0.4159,  ..., -0.0035,  0.7000, -0.0332],
        [-0.2544,  0.6860,  0.3276,  ..., -0.1532, -0.1399, -0.9984],
        ...,
        [-0.2388,  0.5144,  0.4716,  ..., -0.2022, -0.1116, -0.9988],
        [-0.0777,  0.6935,  0.4020,  ..., -0.1393, -0.0520,  0.9999],
        [-0.1248,  0.6928,  0.3466,  ..., -0.1985, -0.0585,  0.9984]],
       device='cuda:0') tensor([[ 0.3166,  0.1811,  0.3994,  ..., -0.1740, -0.1047, -0.9776],
        [-0.0070,  0.7628,  0.4204,  ..., -0.0136,  0.6979, -0.0176],
        [-0.2372,  0.6677,  0.3087,  ..., -0.1396, -0.1096, -0.9919],
        ...,
        [-0.2549,  0.4944,  0.4607,  ..., -0.2152, -0.1289, -0.9886],
        [-0.1011,  0.6765,  0.3921,  ..., -0.1398, -0.0650,  0.9975],
        [-0.1407,  0.6825,  0.3511,  ..., -0.1975, -0.0548,  0.9930]],
       device='cuda:0')
tensor([[-0.0757,  0.7270,  0.3946,  ..., -0.1120, -0.0565,  0.9994],
        [ 0.01

tensor([[-9.4460e-02,  6.9221e-01,  4.1206e-01, -9.9955e-01, -4.7567e-02,
         -8.5001e-02, -9.9892e-01],
        [-7.3949e-02,  5.3949e-01,  4.2224e-01, -1.0000e+00, -2.4208e-01,
         -6.7055e-02,  9.9417e-01],
        [-2.2893e-02,  7.3435e-01,  3.7568e-01, -9.8928e-01, -6.3043e-02,
         -9.1972e-01,  6.8135e-02],
        [-1.7402e-01,  5.9267e-01,  4.3024e-01, -9.9970e-01, -2.4606e-01,
         -2.7171e-02,  9.8706e-01],
        [-3.6194e-02,  6.7045e-01,  4.2937e-01, -9.9978e-01, -9.8840e-02,
         -6.1034e-02,  9.9174e-01],
        [ 4.1229e-02,  7.4478e-01,  4.1417e-01, -9.7776e-01,  1.1472e-02,
          8.9646e-01, -7.4324e-03],
        [-2.1037e-01,  7.1532e-01,  5.9645e-01, -9.8678e-01, -6.3721e-02,
          9.3406e-01, -9.8043e-02],
        [ 5.0764e-02,  7.2044e-01,  4.0315e-01, -9.9955e-01,  9.6476e-01,
         -1.2108e-01, -1.7168e-02],
        [ 7.0311e-01,  6.9220e-01,  3.8847e-01,  9.9944e-01, -1.9006e-03,
         -7.7505e-01,  4.8180e-02],
        [-

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 155/155 [00:08<00:00, 19.24it/s]

0.9581339909652491
148.5107685996136


In [35]:
[[1,2,3]]*10

[[1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3]]

In [3]:
model = LSTMRegressor(input_size=19, output_size=7)

In [4]:
model.load_state_dict(torch.load(r'C:\Users\CSCI-538-HP-Z240-lll\Desktop\UnityProjects\XR-Interaction-Toolkit-Examples\ML\models\lstm_alldata_run1_continue2\checkpoints\model_779.pth'))

<All keys matched successfully>

In [5]:
model.eval()

LSTMRegressor(
  (lstm_1): LSTM(19, 100, batch_first=True)
  (fc1): Linear(in_features=1000, out_features=512, bias=True)
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=256, out_features=64, bias=True)
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=64, out_features=7, bias=True)
)

In [13]:
model.h1 = model.init_hidden(batch_size=1, device='cuda:0')
dummy_input = torch.randn(1,10,19).to('cuda:0')
model.to('cuda:0')
torch.onnx.export(model,
          dummy_input,
          os.path.join(
                'model_779_eval.onnx'
            ),
          )

In [13]:
dataset = LSTMCSVDataset(root_path='../Data_Exp/Exp0')

In [14]:
model = LSTMRegressor(input_size=19, output_size=7)
model.load_state_dict(torch.load(r'C:\Users\CSCI-538-HP-Z240-lll\Desktop\UnityProjects\XR-Interaction-Toolkit-Examples\ML\models\lstm_alldata_run1_continue2\checkpoints\model_779.pth'))

<All keys matched successfully>

In [52]:
model.h1 = model.init_hidden(1,'cpu')
model.eval()
torch.onnx.export(
    model,
    torch.randn((1,10,19)),
    "lstm_run1_779.onnx"
)

c:\users\csci-538-hp-z240-lll\desktop\unityprojects\xr-interaction-toolkit-examples\venv\lib\site-packages\torch\onnx\symbolic_opset9.py:1941: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn("Exporting a model to ONNX with a batch_size other than 1, " +


In [24]:
import onnx

In [53]:
onnx_model = onnx.load('lstm_run1_779.onnx')

In [28]:
import onnxruntime

In [29]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [54]:
ort_session = onnxruntime.InferenceSession('lstm_run1_779.onnx')

In [55]:
x = dataset[10][0].unsqueeze(0)
print(x.shape)
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

torch.Size([1, 10, 19])


In [56]:
ort_outs

[array([[-0.06021449,  0.75974095,  0.3960508 , -1.0000001 , -0.02968227,
         -0.10297136, -0.9999315 ]], dtype=float32)]

In [57]:
dataset[10][1]

tensor([[-0.0714,  0.7554,  0.3997, -0.9980, -0.0390, -0.1019, -0.9991],
        [-0.0717,  0.7556,  0.4001, -0.9980, -0.0388, -0.1017, -0.9991],
        [-0.0717,  0.7561,  0.4006, -0.9980, -0.0384, -0.1017, -0.9991],
        [-0.0717,  0.7565,  0.4009, -0.9980, -0.0383, -0.1017, -0.9991],
        [-0.0719,  0.7567,  0.4013, -0.9980, -0.0381, -0.1017, -0.9991],
        [-0.0720,  0.7570,  0.4017, -0.9980, -0.0381, -0.1015, -0.9990],
        [-0.0721,  0.7568,  0.4019, -0.9980, -0.0379, -0.1014, -0.9990],
        [-0.0722,  0.7560,  0.4021, -0.9980, -0.0377, -0.1015, -0.9990],
        [-0.0724,  0.7553,  0.4023, -0.9980, -0.0371, -0.1014, -0.9990],
        [-0.0725,  0.7540,  0.4024, -0.9981, -0.0368, -0.1017, -0.9990]])

In [50]:
model.eval()
with torch.set_grad_enabled(False):
    model.h1 = model.init_hidden(1,'cpu')
    out = model.forward(dataset[10][0].unsqueeze(0))

In [51]:
out

tensor([[-0.0602,  0.7597,  0.3961, -1.0000, -0.0297, -0.1030, -0.9999]])

In [46]:
np.testing.assert_allclose(to_numpy(out), ort_outs[0], rtol=1e-3, atol=1e-5)

In [58]:
len(dataset)

395161